<a href="https://colab.research.google.com/github/Jyothiraditya135/Some-Codes/blob/main/AIClubHackathon_XGBoost_mean_fill.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder

In [ ]:
from google.colab import files

print(files.upload())

!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c spaceship-titanic-ai-club

Saving kaggle.json to kaggle (1).json
{'kaggle (1).json': b'{"username":"jyothiraditya135","key":"807503915c9e6d0770b3c78104ada889"}'}
mkdir: cannot create directory ‘/root/.kaggle’: File exists
spaceship-titanic-ai-club.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip /content/spaceship-titanic-ai-club.zip

Archive:  /content/spaceship-titanic-ai-club.zip
replace sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
train_dataset = "/content/train.csv"
test_dataset = "/content/test.csv"

In [ ]:
tr_df = pd.read_csv(train_dataset)
te_df = pd.read_csv(test_dataset)

In [ ]:
tr_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [ ]:
# tr_df_n = tr_df.fillna(method='ffill')
# tr_df_n.head(30)

In [ ]:
import random
from sklearn.impute import SimpleImputer

In [ ]:
def Preprocessor(df, tr_te=0):

  cabin_imputer = SimpleImputer(strategy='constant', fill_value='Missing')
  df['Cabin'] = cabin_imputer.fit_transform(df[['Cabin']])
  def generate_random_cabin_code():
    letters = random.choice('ABCDEFGT')
    letter = random.choice('PS')
    number = random.randint(1, 9)
    return f"{letters}/{number}/{letter}"
  df.loc[df['Cabin'] == 'Missing', 'Cabin'] = df[df['Cabin'] == 'Missing'].apply(lambda row: generate_random_cabin_code(), axis=1)

  def generate_random_boolean():
    return random.choice([True, False])
  bool_imputer = SimpleImputer(strategy='constant', fill_value=generate_random_boolean())
  df['CryoSleep'] = bool_imputer.fit_transform(df[['CryoSleep']])
  df['VIP'] = bool_imputer.fit_transform(df[['VIP']])

  def generate_home():
    homes = ['Europa', 'Earth', 'Mars']
    return random.choice(homes)
  bool_imputer = SimpleImputer(strategy='constant', fill_value=generate_home())
  df['HomePlanet'] = bool_imputer.fit_transform(df[['HomePlanet']])

  def generate_dest():
    homes = ['TRAPPIST-1e', 'PSO J318.5-22', '55 Cancri e']
    return random.choice(homes)
  bool_imputer = SimpleImputer(strategy='constant', fill_value=generate_dest())
  df['Destination'] = bool_imputer.fit_transform(df[['Destination']])

  median_imputer = SimpleImputer(strategy='mean')
  df[['Age', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']] = median_imputer.fit_transform(df[['Age', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']])

  for i in range(len(df)):
    if df['CryoSleep'].values[i] == False:
      df['CryoSleep'].values[i] = 0.0
    else:
      df['CryoSleep'].values[i] = 1.0

  for i in range(len(df)):
    if df['VIP'].values[i] == False:
      df['VIP'].values[i] = 0.0
    else:
      df['VIP'].values[i] = 1.0

  df['p/s'] = df['Cabin'].str[-1]

  df['P/S'] = df['p/s'].apply(lambda x: 1 if x == 'P' else 0)
  df['deck'] = df['Cabin'].str[0]
  df['num'] = df['Cabin'].str[2:-2]
  df['deck'] = pd.Categorical(df['deck']).codes+1
  df['group'] = df['PassengerId'].str[0:4]

  enc1 = OneHotEncoder(handle_unknown='ignore')
  enc1_df = enc1.fit_transform(df[['Destination']])
  df_n1 = pd.DataFrame(enc1_df.toarray(), columns = enc1.categories_[0])
  df = df.join(df_n1)

  enc2 = OneHotEncoder(handle_unknown='ignore')
  enc2_df = enc2.fit_transform(df[['HomePlanet']])
  df_n2 = pd.DataFrame(enc2_df.toarray(), columns = enc2.categories_[0])
  df = df.join(df_n2, rsuffix='_enc')

  df.drop(columns = ['p/s', 'Cabin', 'PassengerId', 'HomePlanet', 'Destination', 'Name'], inplace=True)

  df = df.astype(float)

  if tr_te == 0:
    df['Transported'] = df['Transported'].apply(lambda x: 1 if x == True else 0)
    features = df.drop(columns = ['Transported'])

  # df = df.dropna()
  return df

In [ ]:
train_df = Preprocessor(tr_df, tr_te=0)

In [ ]:
train_df.head(10)

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,P/S,deck,num,group,55 Cancri e,PSO J318.5-22,TRAPPIST-1e,Earth,Europa,Mars
0,0.0,39.0,0.0,0.0,0.0,0.0,0.0,0.000000,0,1.0,2.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
1,0.0,24.0,0.0,109.0,9.0,25.0,549.0,44.000000,1,0.0,6.0,0.0,2.0,0.0,0.0,1.0,1.0,0.0,0.0
2,0.0,58.0,1.0,43.0,3576.0,0.0,6715.0,49.000000,0,0.0,1.0,0.0,3.0,0.0,0.0,1.0,0.0,1.0,0.0
3,0.0,33.0,0.0,0.0,1283.0,371.0,3329.0,193.000000,0,0.0,1.0,0.0,3.0,0.0,0.0,1.0,0.0,1.0,0.0
4,0.0,16.0,0.0,303.0,70.0,151.0,565.0,2.000000,1,0.0,6.0,1.0,4.0,0.0,0.0,1.0,1.0,0.0,0.0
5,0.0,44.0,0.0,0.0,483.0,0.0,291.0,0.000000,1,1.0,6.0,0.0,5.0,0.0,1.0,0.0,1.0,0.0,0.0
6,0.0,26.0,0.0,42.0,1539.0,3.0,0.0,0.000000,1,0.0,6.0,2.0,6.0,0.0,0.0,1.0,1.0,0.0,0.0
7,1.0,28.0,0.0,0.0,0.0,0.0,0.0,304.854791,1,0.0,7.0,0.0,6.0,0.0,0.0,1.0,1.0,0.0,0.0
8,0.0,35.0,0.0,0.0,785.0,17.0,216.0,0.000000,1,0.0,6.0,3.0,7.0,0.0,0.0,1.0,1.0,0.0,0.0
9,1.0,14.0,0.0,0.0,0.0,0.0,0.0,0.000000,1,1.0,2.0,1.0,8.0,1.0,0.0,0.0,0.0,1.0,0.0


In [ ]:
test_df = Preprocessor(te_df, tr_te=1)

In [ ]:
test_df.head(10)

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,P/S,deck,num,group,55 Cancri e,PSO J318.5-22,TRAPPIST-1e,Earth,Europa,Mars
0,1.0,27.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,7.0,3.0,13.0,0.0,0.0,1.0,1.0,0.0,0.0
1,0.0,19.0,0.0,0.0,9.000000,0.0,2823.0,0.0,0.0,6.0,4.0,18.0,0.0,0.0,1.0,1.0,0.0,0.0
2,1.0,31.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,3.0,0.0,19.0,1.0,0.0,0.0,0.0,1.0,0.0
3,0.0,38.0,0.0,0.0,6652.000000,0.0,181.0,585.0,0.0,3.0,1.0,21.0,0.0,0.0,1.0,0.0,1.0,0.0
4,0.0,20.0,0.0,10.0,0.000000,635.0,0.0,0.0,0.0,6.0,5.0,23.0,0.0,0.0,1.0,1.0,0.0,0.0
5,0.0,31.0,0.0,0.0,1615.000000,263.0,113.0,60.0,1.0,6.0,7.0,27.0,0.0,0.0,1.0,1.0,0.0,0.0
6,1.0,21.0,0.0,0.0,439.484296,0.0,0.0,0.0,1.0,2.0,2.0,29.0,1.0,0.0,0.0,0.0,1.0,0.0
7,1.0,20.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,4.0,0.0,32.0,0.0,0.0,1.0,0.0,1.0,0.0
8,1.0,23.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,4.0,0.0,32.0,1.0,0.0,0.0,0.0,1.0,0.0
9,0.0,24.0,0.0,0.0,639.000000,0.0,0.0,0.0,0.0,6.0,7.0,33.0,1.0,0.0,0.0,1.0,0.0,0.0


In [ ]:
targets = train_df['Transported']
features = train_df.drop(columns = 'Transported')

In [ ]:
from sklearn import metrics
from sklearn.metrics import mean_squared_error, precision_score, confusion_matrix, accuracy_score

In [ ]:
def evaluation(y_pred, y_test):

# Print Accuracy, Recall, F1 Score, and Precision metrics.
    print('Accuracy: ' + str(metrics.accuracy_score(y_test, y_pred))) #Accuracy of the model
    print('Recall: ' + str(metrics.recall_score(y_test, y_pred))) #True positives/(true positives + False negatives)
    print('Precision: ' + str(metrics.precision_score(y_test, y_pred))) #True positives/(true positives + false positives)

# Print Confusion Matrix
    print('\nConfusion Matrix:')
    print(' TN,  FP, FN, TP') #True negative, False Positive, False negative, True Positive
    print(confusion_matrix(y_test, y_pred).ravel())

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
fs_tr, fs_val, trgs_tr, trgs_val = train_test_split(features, targets, random_state=1, test_size=.27)

In [ ]:
import numpy as np
from xgboost import XGBClassifier

params = {
    'objective': 'binary:logistic',  # Change to 'multi:softmax' if you have multiclass targets
    'eval_metric': 'logloss',        # Use appropriate evaluation metric for your problem
    'max_depth': 3,                 # Max depth of each tree (tune this parameter)
    'learning_rate': 0.09,           # Step size shrinkage used in update to prevent overfitting
    'n_estimators': 72            # Number of boosting rounds (tune this parameter)
}

history_model = XGBClassifier(**params)

history_model.fit(fs_tr, trgs_tr)
y_pred = history_model.predict(fs_val)
evaluation(y_pred, trgs_val)

Accuracy: 0.8087734241908007
Recall: 0.8439597315436241
Precision: 0.7927501970055162

Confusion Matrix:
 TN,  FP, FN, TP
[ 893  263  186 1006]


In [ ]:
features_test = test_df
test_preds = history_model.predict(features_test)

In [ ]:
data = {
    'PassengerId' : te_df['PassengerId'],
    'Transported' : test_preds
}
Submit_df = pd.DataFrame(data)

In [ ]:
Submit_df['Transported'] = Submit_df['Transported'].apply(lambda x:True if x==1 else False)

In [ ]:
Submit_df

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,False
...,...,...
4272,9266_02,True
4273,9269_01,False
4274,9271_01,True
4275,9273_01,True


In [ ]:
submission_file = 'submissions.csv'
Submit_df.to_csv(submission_file, index=False)